In [56]:
import numpy as np
import scipy.sparse as sp
import tensorflow as tf
import time
import pandas as pd
import pickle
from multiprocessing import Process
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from spektral.data import Dataset, DisjointLoader, Graph
from spektral.layers import GCSConv, GlobalAvgPool, ECCConv
from spektral.transforms.normalize_adj import NormalizeAdj
from spektral.utils import reorder
from ipywidgets import Checkbox, Dropdown, Accordion, VBox
import sklearn.metrics as metrics
from sklearn.calibration import calibration_curve
from spektral.data import Dataset, Graph, DisjointLoader
from spektral.layers import CrystalConv, GlobalAvgPool
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC

import copy
import logging
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd
import random
import sys
import tensorflow as tf
import requests
from os.path import isfile
import time

In [2]:
node_feat = pd.read_pickle('simple_nodes.pkl')

In [54]:
adj_mat = pd.read_pickle('adj_mat.pkl')

In [4]:
edge_feat = pd.read_pickle('edge_features.pkl')

In [5]:
edge_feat = adj_mat * edge_feat

In [100]:
y = pd.read_pickle('y.pkl')

In [101]:
y = y.reset_index()
y['uniqueplayId'] = y['uniqueplayId'].astype(int)
y = y.set_index(['uniqueplayId','frameId'])

In [8]:
node_feat = node_feat.reset_index()

In [9]:
node_feat['uniqueplayId'] = node_feat['uniqueplayId'].astype(int)
node_feat = node_feat.set_index(['uniqueplayId','frameId','nflId'])

In [10]:
node_feat.isna().sum()

new_x                0
new_y                0
Defense              0
frames_after_snap    0
dtype: int64

In [51]:
def make_graph(index):
    ## PLAY_ID MUST BE A STRING
    
    play_id = index[0]
    frame_id = index[1]

    # Node features
    ## filter the node features matrix by given play id and frame id
    x_temp = node_feat.query(f'(uniqueplayId=={play_id})&(frameId=={frame_id})')
    x_temp = np.array(x_temp)
    #print(x_temp.shape)

    # Adjacency
    a_temp = adj_mat[(play_id, frame_id)]
    a_temp = sp.csr_matrix(a_temp)
    #print(a_temp.shape)
    
    # Edges
    ## get the correct edge matrix based on play id and frame id
    e_temp = edge_feat[(play_id, frame_id)]
    e_sp_mat = sp.find(e_temp)

    edge_indeces = np.array([e_sp_mat[0], e_sp_mat[1]]).T

    edge_vals = e_sp_mat[2]
    e_temp = reorder(edge_indeces, edge_features=edge_vals)[1].reshape(len(edge_vals), 1)
    #print(e_temp.shape)

    # Labels
    ## get the single label of coverage from y for that play id
    y_temp = y.query(f'(uniqueplayId=={play_id})&(frameId=={frame_id})').values[0]
    #print(y_temp.shape)

    return Graph(x=x_temp, a=a_temp, e=e_temp, y=y_temp)

In [52]:
make_graph(('202109090097', 6.0))

Graph(n_nodes=23, n_node_features=4, n_edge_features=1, n_labels=7)

In [53]:
%%time
################################################################################
# Load data
################################################################################

from concurrent.futures import ProcessPoolExecutor

class MyDataset(Dataset):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def read(self):
        
        all_graphs = []
        indeces = adj_mat.index

        with ProcessPoolExecutor(max_workers=40) as executor:
            for r in executor.map(make_graph, indeces):
                all_graphs.append(r)
            # We must return a list of Graph objects
        return all_graphs


data = MyDataset()

CPU times: user 2min 40s, sys: 54.9 s, total: 3min 35s
Wall time: 37min 33s


In [54]:
with open('simple_graph_data_w_edges.pkl', 'wb') as b:
    pickle.dump(data, b)

## Instead going to seperate out the last few plays first and make that our test data

In [215]:
class MyDataset(Dataset):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def read(self):
        
        all_graphs = []
        indeces = edge_feat.index

        with ProcessPoolExecutor(max_workers=40) as executor:
            for r in executor.map(make_graph, indeces):
                all_graphs.append(r)
            # We must return a list of Graph objects
        return all_graphs
    
data = pd.read_pickle('simple_graph_data_w_edges.pkl')

In [216]:
test_data = data[150959:]

In [217]:
data = data[:150959]

In [163]:
# Train/valid/test split
idxs = np.random.permutation(len(data))
split_va = int(0.9 * len(data))
idx_tr, idx_va = np.split(idxs, [split_va])
data_tr = data[idx_tr]
data_va = data[idx_va]

In [171]:
learning_rate = 0.01  # Learning rate
epochs = 400  # Number of training epochs
es_patience = 10  # Patience for early stopping
batch_size = 32  # Batch size
layers = 3 # Number of CrystalConv layers
channels = 128 # Number of hidden nodes
n_out = 7

In [172]:
# Data loaders
loader_tr = DisjointLoader(data_tr, batch_size=batch_size, epochs=epochs)
loader_va = DisjointLoader(data_va, batch_size=batch_size)
loader_te = DisjointLoader(test_data, batch_size=batch_size)

In [173]:
class GNN(Model):
    '''
    Building the Graph Neural Network configuration with Model as the parent class 
    from spektral library.
    '''
    def __init__(self, n_layers):
        '''
        Constructor code for setting up the layers needed for training the model.
        '''
        super().__init__()
        self.conv1 = CrystalConv()
        self.convs = []
        for _ in range(1, n_layers):
            self.convs.append(
                CrystalConv()
            )
        self.pool = GlobalAvgPool()
        self.dense1 = Dense(channels, activation = tf.keras.layers.LeakyReLU(alpha = 0.1))
        self.dropout = Dropout(0.5)
        self.dense2 = Dense(channels, activation = tf.keras.layers.LeakyReLU(alpha = 0.1))
        self.dense3 = Dense(n_out, activation="softmax")

    def call(self, inputs):
        '''
        Build the neural network.
        '''
        x, a, e, i = inputs
        x = self.conv1([x, a, e])
        for conv in self.convs:
            x = conv([x, a, e])
        x = self.pool([x, i])
        x = self.dense1(x)
        x = self.dropout(x)
        x = self.dense2(x)
        x = self.dropout(x)
        return self.dense3(x)
    
model = GNN(layers)
optimizer = Adam(learning_rate=learning_rate)
loss_fn = CategoricalCrossentropy()

In [174]:
def evaluate(loader):
    output = []
    step = 0
    while step < loader.steps_per_epoch:
        step += 1
        inputs, target = loader.__next__()
        pred = model(inputs, training=False)
        #print(target)
        #print(inputs)
        #print(pred)
        outs = (
            loss_fn(target, pred),
            tf.reduce_mean(categorical_accuracy(target, pred)),
            len(target),  # Keep track of batch size
        )
        output.append(outs)
        if step == loader.steps_per_epoch:
            output = np.array(output)
            return np.average(output[:, :-1], 0, weights=output[:, -1])

In [55]:
@tf.function(input_signature=loader_tr.tf_signature(), experimental_relax_shapes=True)
def train_step(inputs, target):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(target, predictions) + sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    acc = tf.reduce_mean(categorical_accuracy(target, predictions))
    return loss, acc

In [56]:
%%time

epoch = step = 0
best_val_loss = np.inf
best_weights = None
patience = es_patience
results = []
for batch in loader_tr:
    step += 1
    loss, acc = train_step(*batch)
    results.append((loss, acc))
    if step == loader_tr.steps_per_epoch:
        step = 0
        epoch += 1

        # Compute validation loss and accuracy
        val_loss, val_acc = evaluate(loader_va)
        print(
            "Ep. {} - Loss: {:.3f} - Acc: {:.3f} - Val loss: {:.3f} - Val acc: {:.3f}".format(
                epoch, *np.mean(results, 0), val_loss, val_acc
            )
        )

        # Check if loss improved for early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience = es_patience
            print("New best val_loss {:.3f}".format(val_loss))
            best_weights = model.get_weights()
        else:
            patience -= 1
            if patience == 0:
                print("Early stopping (best val_loss: {})".format(best_val_loss))
                break
        results = []

/home/can2hr/.local/lib/python3.9/site-packages/spektral/data/utils.py:221: UserWarning: you are shuffling a 'MyDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)


Ep. 1 - Loss: 1.709 - Acc: 0.338 - Val loss: 1.546 - Val acc: 0.379
New best val_loss 1.546
Ep. 2 - Loss: 1.590 - Acc: 0.353 - Val loss: 1.535 - Val acc: 0.384
New best val_loss 1.535
Ep. 3 - Loss: 1.615 - Acc: 0.348 - Val loss: 1.531 - Val acc: 0.387
New best val_loss 1.531
Ep. 4 - Loss: 1.511 - Acc: 0.395 - Val loss: 1.419 - Val acc: 0.437
New best val_loss 1.419
Ep. 5 - Loss: 1.510 - Acc: 0.399 - Val loss: 1.420 - Val acc: 0.423
Ep. 6 - Loss: 1.438 - Acc: 0.429 - Val loss: 1.335 - Val acc: 0.470
New best val_loss 1.335
Ep. 7 - Loss: 1.471 - Acc: 0.437 - Val loss: 1.322 - Val acc: 0.477
New best val_loss 1.322
Ep. 8 - Loss: 1.538 - Acc: 0.435 - Val loss: 1.628 - Val acc: 0.329
Ep. 9 - Loss: 1.446 - Acc: 0.422 - Val loss: 1.345 - Val acc: 0.463
Ep. 10 - Loss: 1.376 - Acc: 0.454 - Val loss: 1.309 - Val acc: 0.490
New best val_loss 1.309
Ep. 11 - Loss: 1.370 - Acc: 0.463 - Val loss: 1.314 - Val acc: 0.483
Ep. 12 - Loss: 1.677 - Acc: 0.404 - Val loss: 1.333 - Val acc: 0.489
Ep. 13 - Loss

In [ ]:
model = tf.keras.models.load_model('saved_model/simple_best_model', compile = False)

In [57]:
model.set_weights(best_weights)  # Load best model
test_loss, test_acc = evaluate(loader_te)
print("Done. Test loss: {:.4f}. Test acc: {:.2f}".format(test_loss, test_acc))

Done. Test loss: 1.2245. Test acc: 0.54


In [218]:
loader_te = DisjointLoader(test_data, batch_size=batch_size, shuffle = False)

In [221]:
test_loss, test_acc = evaluate(loader_te)
print("Done. Test loss: {:.4f}. Test acc: {:.2f}".format(test_loss, test_acc))

Done. Test loss: 1.1955. Test acc: 0.55


In [181]:
def evaluate_play(loader):
    true = []
    predict = []
    step = 0
    while step < loader.steps_per_epoch:
        step += 1
        inputs, target = loader.__next__()
        pred = model(inputs, training=False)
        true.append(target)
        predict.append(pred)
    return true, predict

In [219]:
true, predict = evaluate_play(loader_te)

In [222]:
predictions = pd.DataFrame()
for batch in predict:
    for i in batch:
        pred = pd.DataFrame(i).T
        predictions = pd.concat([predictions, pred])

In [223]:
predictions['predicted_coverage'] = predictions.idxmax(axis = 1)

In [224]:
coverage = pd.DataFrame(y[150959:].idxmax(axis=1)).rename(columns = {0:'coverage'})

In [225]:
coverage['predicted_coverage'] = predictions['predicted_coverage'].values

In [226]:
sum(coverage['predicted_coverage'] == coverage['coverage'])/len(coverage)

0.5508479067302596

In [227]:
## Should be 55% and expected to be higher once we look by play at the most predicted one

In [229]:
coverage = coverage.reset_index()

In [230]:
coverage

,uniqueplayId,frameId,coverage,predicted_coverage
0,20211017043813,6.0,3,3
1,20211017043813,7.0,3,3
2,20211017043813,8.0,3,3
3,20211017043813,9.0,3,3
4,20211017043813,10.0,3,3
...,...,...,...,...
37735,2021102500933,34.0,4,3
37736,2021102500933,35.0,4,4
37737,2021102500933,36.0,4,4
37738,2021102500933,37.0,4,3


In [290]:
counts = pd.DataFrame(coverage.groupby(['uniqueplayId','predicted_coverage']).size()).rename(columns = {0:'count'})

In [293]:
counts = counts.loc[counts.groupby(['uniqueplayId'])["count"].idxmax()]

In [298]:
counts = counts.reset_index()

In [299]:
results = pd.DataFrame(coverage.groupby('uniqueplayId').first()['coverage']).reset_index()

In [301]:
results = results.merge(counts, how = 'left')

In [302]:
results

,uniqueplayId,coverage,predicted_coverage,count
0,202110170483,3,3,15
1,202110170673,3,4,15
2,202110170762,1,1,22
3,202110170856,3,3,18
4,202110170955,6,6,8
...,...,...,...,...
1415,20211025003684,1,1,31
1416,20211025003735,3,1,17
1417,20211025003904,4,4,23
1418,20211025003926,2,6,15


In [304]:
sum(results['predicted_coverage'] == results['coverage'])/len(results)

0.5823943661971831

In [305]:
## Overall 58% accuracy